In [1]:
%%sql
WITH store_days AS (
SELECT DISTINCT
       store_key,
       TO_DATE(CAST(date_id AS STRING), 'yyyyMMdd') AS d          -- 20190602 → 2019-06-02
FROM   Gold_Data.fact_sales.fact_sales),
numbered AS (
    SELECT
        store_key,
        d,
        ROW_NUMBER() OVER (PARTITION BY store_key ORDER BY d)                 AS rn,
        DATEDIFF(d, '1970-01-01') - rn                                        AS grp_marker
    FROM store_days
)
SELECT
    store_key,
    MIN(d)  AS span_start,
    MAX(d)  AS span_end,
    COUNT(*) AS span_len
FROM  numbered
GROUP BY store_key, grp_marker
ORDER BY store_key, span_start;


StatementMeta(, 0e35115f-28cb-487e-b1c5-dbf626f7d7e2, 2, Finished, Available, Finished)

<Spark SQL result set with 66 rows and 4 fields>

In [3]:
%%sql
WITH store_days AS (
SELECT DISTINCT
       store_key,
       TO_DATE(CAST(date_id AS STRING), 'yyyyMMdd') AS d          -- 20190602 → 2019-06-02
FROM   Gold_Data.fact_sales.fact_sales),
numbered AS (
    SELECT
        store_key,
        d,
        ROW_NUMBER() OVER (PARTITION BY store_key ORDER BY d)                 AS rn,
        DATEDIFF(d, '1970-01-01') - rn                                        AS grp_marker
    FROM store_days
),
store_active_period as (
SELECT
    store_key,
    MIN(d)  AS span_start,
    MAX(d)  AS span_end,
    COUNT(*) AS span_len
FROM  numbered
GROUP BY store_key, grp_marker
ORDER BY store_key, span_start)
SELECT store_key, count(*) as number_of_records FROM store_active_period GROUP BY store_key

StatementMeta(, 0e35115f-28cb-487e-b1c5-dbf626f7d7e2, 4, Finished, Available, Finished)

<Spark SQL result set with 63 rows and 2 fields>

From this we conclude that every store is opened all year long for the exception of store-key 60